In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 10000
N_bound = 200

la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)

bound_x = -1+(1-(-1))*lhs(1,N_bound)

bound_t = 0+(1-(0))*lhs(1,N_bound)

# #x,t  #u,v
X_train = torch.from_numpy(traindata).float()
# x_bound = torch.from_numpy(bound_x).float()
# t_bound = torch.from_numpy(bound_t).float()
x = X_train[:,0:1]
t = X_train[:,1:2]

x.requires_grad_()
t.requires_grad_()

###########GPU###########
# x_bound = x_bound.cuda()
# t_bound = t_bound.cuda()
x = x.cuda()
t = t.cuda()
###########GPU###########

In [ ]:
import scipy.io
data = scipy.io.loadmat('./burgers_shock.mat')
t_ = data['t'].flatten()[:,None]
x_ = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x_,t_)

X_star = np.hstack((X.flatten()[:,None],T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]  

In [ ]:
X_star = torch.from_numpy(X_star).float().cuda()

In [ ]:
np.random.seed(1234)
n=200

index_t = (lhs(1,n))*len(t_)
index_t = np.floor(index_t).reshape(n,).astype(int)
t_star = t_[index_t]

index_x = (lhs(1,n))*len(x_)
index_x = np.floor(index_x).reshape(n,).astype(int)
x_star = x_[index_x]

x_t_star = np.hstack((x_star,t_star))
u_star_ = Exact[index_t,index_x]

In [ ]:
observe_data = x_t_star
observe_u = u_star_.reshape(-1,1)
observe_data = torch.from_numpy(observe_data).float()
observe_u = torch.from_numpy(observe_u).float()

In [ ]:
C1 = torch.tensor(0.5, requires_grad=True)

In [ ]:
###########GPU###########
observe_u = observe_u.cuda()
observe_data = observe_data.cuda()
###########GPU###########

In [ ]:
np.random.seed(5678)
index_t = (lhs(1,10000))*len(t_)
index_t = index_t.astype(int).reshape(10000,)
t_star = t_[index_t]
index_x = (lhs(1,10000))*len(x_)
index_x = index_x.astype(int).reshape(10000,)
x_star = x_[index_x]
test_data = np.hstack((x_star,t_star))
test_u = Exact[index_t,index_x].reshape(-1,1)

test_data = torch.from_numpy(test_data).float()
test_u = torch.from_numpy(test_u).float()

###########GPU###########
test_data = test_data.cuda()
test_u = test_u.cuda()
###########GPU###########

In [ ]:
def relative_l2(u_pred, u_real):

    l2 = torch.norm(u_real - u_pred, p=2) / torch.norm(u_real, p=2)
    
    return l2.item()  # Convert the result back to a Python float

In [ ]:
def output_transform(x, y):
    x_in = x[:, 0:1]
    t_in = x[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * y - torch.sin(np.pi * x_in)

In [ ]:
def Loss_f(x_inside,y_inside,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_inside,y_inside),1))
    E_inside = output_transform(torch.cat((x_inside,y_inside),axis=1), E_inside)
    
    
    E_x = autograd.grad(outputs=E_inside, inputs=x_inside,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_xx = autograd.grad(outputs=E_x, inputs=x_inside,
                              grad_outputs=torch.ones_like(E_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_t = autograd.grad(outputs=E_inside, inputs=y_inside,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]     

    #loss_f = torch.mean(torch.square(E_t+E_inside*E_x-C1*(E_xx)))
    #loss_f = torch.mean(torch.square(deata_E+torch.sin(torch.tensor(np.pi)*x)*torch.sin(torch.tensor(np.pi)*y)))
    ########### loss f  ###########
    
    if return_sequence=='yes':
        return torch.square(E_t+E_inside*E_x-C*(E_xx))
    else:
        return torch.mean(torch.square(E_t+E_inside*E_x-C*(E_xx)))

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA on GPU")
else:
    device = "cpu"
    print("Using CPU")

## PINN

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


##############PW_L_f###############
e_k = 0.005
q_k = 0.0005
s_k = 0.7
Ada_CL_L_f = Ada_CL(e_k,q_k,s_k,device)
weight_sequence_L_f = (1/N_train)*np.ones((N_train,1))
##############PW_L_f###############


PINNs1 = NN_H2(2, 20, 7, 1)
PINNs1.cuda()

#PINNs1.apply(weights_init)
import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

loss_all_1 = []
loss_all_2 = []
loss_T_1 = []
test_loss_1 = []
C1_list = []
beta_number_list = []
rho_k_list = []

loss_f_sequence_list = []
weight_sequence_L_f_list = []
plot_heat_list = []

nIter1 = 10000

loss1_value = 1
it = 0
while it<nIter1:
        
    ##############PW_L_f###############
    loss_f_sequence = Loss_f(x,t,PINNs1,C1,return_sequence='yes')
    loss_f,weight_sequence_L_f,rho_k,beta_sequence  = Ada_CL_L_f.Training_Scheduler_torch(loss_f_sequence, weight_sequence_L_f,alpha_k_function = 'default',record = 'yes')
    beta_number_list.append(np.sum(beta_sequence)/N_train)
    rho_k_list.append(rho_k)
    ##############PW_L_f###############    

    ##### loss observation  ######        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data, E_observation)    
    loss_observation = torch.mean(torch.square(E_observation-observe_u))         
    ##### loss observation  ######       
    
    #####loss PI#######
    loss = loss_f+10*loss_observation 
    #####loss PI#######
    
    #########  test_loss NRMSE  #########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data, pre_u)      
    test_loss = relative_l2(pre_u,test_u)
    #########  test_loss NRMSE  #########
           

    ############## Record ###############
    test_loss_1.append(test_loss)
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation.item()) 
    loss_all_1.append(loss.item())
    ############## Record ###############
    
    ############## Record Original ###############
    loss_f_original = Loss_f(x,t,PINNs1,C1,return_sequence='not')
    loss_original = loss_f_original+10*loss_observation 
    loss_all_2.append(loss_original.item())
    ############## Record Original ###############
    
    if it % 1000 == 0:
        loss_f_sequence_list.append(loss_f_sequence.cpu().detach().numpy())
        weight_sequence_L_f_list.append(weight_sequence_L_f)
        pre_u_for_plot = PINNs1(X_star)
        pre_u_for_plot = output_transform(X_star, pre_u_for_plot)     
        plot_heat_list.append(pre_u_for_plot.cpu().detach().numpy())
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss) 
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    it = it + 1        
    

print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)